<a href="https://colab.research.google.com/github/2303A52261/Generative-AI/blob/main/2303A52261_week6_ASS6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. (1 ponto) Design a multilayer ANN architecture according to the requirements shown below.
Train, test, save (.h5) and deploy the model to predict the housing price using Keras deep
learning library
2. (1 ponto) Calculate training and testing error metrics
3. (1 ponto) Build the application by loading the saved ANN model.
Tabela 1: ANN Architecture
Layer Neurons Activation Function
Hidden Layer - 1 15 tanh
Hidden Layer - 2 20 tanh
Hidden Layer - 3 15 tanh
Tabela 2: Training Parameters
loss function epochs batch size error metric Optimizer
Mean Square Error 100 16 Mean Square Error SGD
Dataset: https://drive.google.com/file/d/1AcdENlVm5dccNyo_vgdMbneX8YVvH5R3/view?
usp=sharing
• Expected learning Outcomes from this assignment related to python
– Students are able to build ANN model with python deep learning libraries
– Students are able to deploy trained ANN model
– Students are able to measure training and testing performance of trained model

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Load dataset (Assuming CSV format after extracting from Google Drive)
url = '/content/Housing.csv'  # Replace this with the actual dataset path
df = pd.read_csv(url)

# Split the dataset into input features and target variable (housing prices)
X = df.drop('price', axis=1)  # Assuming 'price' is the target variable
y = df['price']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical features (columns with object dtype)
categorical_features = X_train.select_dtypes(include=['object']).columns

# Create and fit OneHotEncoder for categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for numpy array
encoded_features_train = encoder.fit_transform(X_train[categorical_features])
encoded_features_test = encoder.transform(X_test[categorical_features])

# Create DataFrames for encoded features
encoded_df_train = pd.DataFrame(encoded_features_train, columns=encoder.get_feature_names_out(categorical_features), index=X_train.index)
encoded_df_test = pd.DataFrame(encoded_features_test, columns=encoder.get_feature_names_out(categorical_features), index=X_test.index)

# Drop original categorical columns and concatenate encoded features
X_train = X_train.drop(categorical_features, axis=1)
X_test = X_test.drop(categorical_features, axis=1)
X_train = pd.concat([X_train, encoded_df_train], axis=1)
X_test = pd.concat([X_test, encoded_df_test], axis=1)


# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanSquaredError

# Define the ANN model
model = Sequential()

# Hidden Layer 1
model.add(Dense(15, input_dim=X_train.shape[1], activation='tanh'))

# Hidden Layer 2
model.add(Dense(20, activation='tanh'))

# Hidden Layer 3
model.add(Dense(15, activation='tanh'))

# Output layer (Assuming it's a regression problem, one neuron for price prediction)
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=SGD(),
              metrics=[MeanSquaredError()])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=(X_test, y_test))


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 10110192582656.0000 - mean_squared_error: 10110192582656.0000 - val_loss: 5237092384768.0000 - val_mean_squared_error: 5237092384768.0000
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2979877617664.0000 - mean_squared_error: 2979877617664.0000 - val_loss: 5481504440320.0000 - val_mean_squared_error: 5481504440320.0000
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3125653798912.0000 - mean_squared_error: 3125653798912.0000 - val_loss: 5268454768640.0000 - val_mean_squared_error: 5268454768640.0000
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2839194894336.0000 - mean_squared_error: 2839194894336.0000 - val_loss: 5056722632704.0000 - val_mean_squared_error: 5056722632704.0000
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2978483011584.0000 - mean_squared_error: 2978483011584.0000 - val_loss: 5240757157888.0000 - val_mean_squared_error: 5240757157888.0000
Epoch 6/100
28/28 ━━━━

In [4]:
# Calculate the training error
train_loss, train_mse = model.evaluate(X_train, y_train)
print(f"Training MSE: {train_mse}")

# Calculate the testing error
test_loss, test_mse = model.evaluate(X_test, y_test)
print(f"Testing MSE: {test_mse}")


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3201359413248.0000 - mean_squared_error: 3201359413248.0000 
Training MSE: 3152620290048.0
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 5272115347456.0000 - mean_squared_error: 5272115347456.0000 
Testing MSE: 5056006455296.0


In [5]:
# Save the trained model
model.save('housing_price_model.h5')


In [6]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('housing_price_model.h5')

# Make predictions on the test data
predictions = loaded_model.predict(X_test)

# Example: Compare predictions with actual values
for actual, predicted in zip(y_test[:10], predictions[:10]):
    print(f"Actual: {actual}, Predicted: {predicted[0]}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Actual: 4060000, Predicted: 4969639.0
Actual: 6650000, Predicted: 4969639.0
Actual: 3710000, Predicted: 4969639.0
Actual: 6440000, Predicted: 4969639.0
Actual: 2800000, Predicted: 4969639.0
Actual: 4900000, Predicted: 4969639.0
Actual: 5250000, Predicted: 4969639.0
Actual: 4543000, Predicted: 4969639.0
Actual: 2450000, Predicted: 4969639.0
Actual: 3353000, Predicted: 4969639.0
